In [8]:
from typing import Dict,TypedDict , List
from langgraph.graph import StateGraph,START,END
import random

In [9]:
class AgentState(TypedDict):
    player_name: str
    guesses: list
    attempts: int
    lower_bound: int
    upper_bound: int
    target: int
    last_guess: int
    hint: str

MAX_ATTEMPTS = 7

In [10]:
def setup(state: AgentState) -> AgentState:
    state.setdefault("guesses", [])
    state.setdefault("attempts", 0)
    state.setdefault("lower_bound", 1)
    state.setdefault("upper_bound", 20)

    if "target" not in state:
        state["target"] = random.randint(state["lower_bound"], state["upper_bound"])

    state["last_guess"] = None
    state["hint"] = ""
    return state

def make_guess(state: AgentState) -> AgentState:
    lo = state["lower_bound"]
    hi = state["upper_bound"]

    if lo > hi:
        guess = lo
    else:
        guess = random.randint(lo, hi)

    state["last_guess"] = guess
    state["guesses"].append(guess)
    state["attempts"] += 1
    return state

def hint_node(state: AgentState) -> AgentState:
    guess = state["last_guess"]
    target = state["target"]

    if guess < target:
        state["hint"] = "higher"
        state["lower_bound"] = guess + 1
    elif guess > target:
        state["hint"] = "lower"
        state["upper_bound"] = guess - 1
    else:
        state["hint"] = "correct"

    return state

def decider(state: AgentState) -> str:
    if state["hint"] == "correct":
        return "end"
    if state["attempts"] >= MAX_ATTEMPTS:
        return "end"
    return "continue"

In [11]:
graph = StateGraph(AgentState)

graph.add_node("setup", setup)
graph.add_node("guess", make_guess)
graph.add_node("hint_node", hint_node)

graph.add_edge(START, "setup")
graph.add_edge("setup", "guess")
graph.add_edge("guess", "hint_node")
graph.add_conditional_edges("hint_node", decider,
                            {"continue": "guess", "end": END})

app = graph.compile()

In [16]:
ans=app.invoke({
    "player_name": "The Rock",
    "guesses": [],
    "attempts": 0
})
print(ans)
print(ans["hint"])

{'player_name': 'The Rock', 'guesses': [10, 14, 11, 13, 12], 'attempts': 5, 'lower_bound': 12, 'upper_bound': 12, 'target': 12, 'last_guess': 12, 'hint': 'correct'}
correct
